# Convert a csv database into posgres

This follows the notebook: ../general-docs/python_sql_dev_setups.ipynb


In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [2]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'autism-docs'
username = 'rangel'

In [3]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print (engine.url)

postgres://rangel@localhost/autism-docs


In [4]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [5]:
# read a database from CSV and load it into a pandas dataframe
df = pd.DataFrame.from_csv('articles-n-forums-posts.csv')
df.head(2)

,category,href,source,text,title,tokens
0,['category-autism-therapy'],https://www.autismparentingmagazine.com/autism...,https://www.autismparentingmagazine.com/,For children with autism spectrum disorder (AS...,"Autism, Head Banging and other Self Harming Be...","['for', 'children', 'with', 'autism', 'spectru..."
1,['category-autism-therapy'],https://www.autismparentingmagazine.com/high-q...,https://www.autismparentingmagazine.com/,Dr. Stephen Shore once said “If you’ve met one...,High Quality ABA Treatment: What Every Parent...,"['dr', 'stephen', 'shore', 'once', 'said', 'if..."


In [6]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
df.to_sql('articles-n-forums-posts', engine, if_exists='replace')

In [7]:
## Now try the same queries, but in python!

# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM \"articles-n-forums-posts\" WHERE title LIKE '%autism%';
"""
query = pd.read_sql_query(sql_query,con)

query.head()

,index,category,href,source,text,title,tokens
0,21,['category-autism-and-diet'],https://www.autismparentingmagazine.com/sugary...,https://www.autismparentingmagazine.com/,Sivakumari asks: My daughter loves to eat ice-...,Q and A Do sugary foods make children with aut...,"['sivakumari', 'asks', 'my', 'daughter', 'love..."
1,269,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n It is hard at so young age to conclu...,is my 2years son normal or he have autism,"['it', 'is', 'hard', 'at', 'so', 'young', 'age..."
2,238,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,"\n From what I can tell, what you descr...",is my 2 year old son has sign of autism?,"['from', 'what', 'i', 'can', 'tell', 'what', '..."
3,267,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,"\n At this point, it is not possible to...",does the e.e.g confirm if the baby has autism,"['at', 'this', 'point', 'it', 'is', 'not', 'po..."
4,271,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n Sounds like post-traumatic stress di...,I am diagnosed with autism but id it true...?,"['sounds', 'like', 'post', 'traumatic', 'stres..."
